In [1]:
import hashlib
import os
import secrets
import warnings
from tabulate import tabulate
from collections import defaultdict
import galois

warnings.filterwarnings("ignore")

## Explanation of the `StatelessConsistentHash` Class

This Python class implements a hash function that consistently maps any given input item to one of two values, `+1` or `-1`. It is designed to simulate a **fully independent** random choice for each unique item while being **stateless** (requiring no memory of past items) and **consistent** (always producing the same output for the same input).

---

### 1. Initialization (`__init__` method)

The setup for the hash function happens when an object of the class is created.

```python
def __init__(self):
    """
    In os.urandom, the number of bytes should be chosen to be 16 or higher because eight is obsolete.
    """
    self.seed = os.urandom(16)
```

* **`self.seed = os.urandom(16)`**: This is the most critical part of the setup.
    * `os.urandom(16)` generates 16 bytes of cryptographically strong random data provided by the operating system. This is not pseudo-random; it's suitable for security-sensitive applications.
    * This **seed** is generated **once** per object instance and stored. Its purpose is to make this specific hash function unique. Two different `StatelessConsistentHash` objects will have different seeds, and therefore, will produce different hash values for the same input item.

---

### 2. The Hashing Process (`hash` method)

This method takes an `item` and returns its corresponding `+1` or `-1` value. The process can be broken down into four main steps.

#### Step 1: Input Serialization

```python
item_bytes = str(item).encode('utf-8')
```

The input `item` is first converted into a standardized byte sequence. This ensures that any type of input (integer, string, etc.) is handled consistently before being passed to the cryptographic hash function.

#### Step 2: Hashing with SHA-256

```python
hasher = hashlib.sha256()
hasher.update(self.seed)
hasher.update(item_bytes)
```

* **`hashlib.sha256()`**: This initializes a standard, secure cryptographic hash object.
* **Avalanche Effect**: SHA-256 guarantees that a tiny change in the input (even one bit) will result in a completely different and unpredictable output hash. This property is what allows us to treat the output as random.
* **`hasher.update(...)`**: The `update` method mixes data into the hash.
    1.  First, the object's unique `self.seed` is added.
    2.  Second, the `item_bytes` are added.

    By combining a unique seed with the item, we ensure that the final hash is a unique and deterministic product of both the object instance and the specific item.

#### Step 3: Conversion to an Integer

```python
hash_digest = hasher.digest()
hash_int = int.from_bytes(hash_digest, 'big')
```

* **`hasher.digest()`**: This returns the final hash value as a 32-byte string (for SHA-256).
* **`int.from_bytes(...)`**: This standard method converts the raw byte string into a single, very large integer. This makes it easy to perform mathematical operations on the hash result.

#### Step 4: Mapping to $\{-1, +1\}$

```python
if hash_int % 2 == 0:
    return 1
else:
    return -1
```

This is the final step where the large random integer is mapped to either `+1` or `-1`.
* The logic `hash_int % 2` simply checks the last bit of the integer to see if it's even or odd.
* Because the output of SHA-256 is uniformly distributed, its last bit is also expected to be uniformly random, with a 50% chance of being 0 or 1. This makes it an excellent and simple way to derive a fair coin flip from the hash.

---

In [2]:
class StatelessConsistentHash:
    def __init__(self):

        """
        In os.urandom, the number of bytes should be chosen to be 16 or higher because eight is obsolete.
        """
        self.seed = os.urandom(16)

    def hash(self, item):

        item_bytes = str(item).encode('utf-8')

        hasher = hashlib.sha256()
        hasher.update(self.seed)
        hasher.update(item_bytes)

        """
        The sha256 protocol guarantees that small changes in the input cause large changes in the output, and hence, given this assumption, the bits can be considered completely independent and random.
        """
        """
        It guarantees that the "hasher" object produces unique results. That is, on each system, the output is different and completely random, depending on the seed.
        """

        hash_digest = hasher.digest()

        hash_int = int.from_bytes(hash_digest, 'big')

        if hash_int % 2 == 0:
            return 1
        else:
            return -1

## Mathematical Concept of Generating 4-wise Independent Bits

Assume we want to generate $n$ 4-wise independent values.

**1. Choose a Finite Field:**
First, we choose a prime number $p$ such that it is greater than or equal to $n$ ($p \ge n$). All of our calculations will be performed in the finite field $GF(p)$. This means all addition and multiplication operations are done modulo $p$ (modular arithmetic).

**2. Create a Random Polynomial:**
We construct a degree-3 polynomial as follows:
$$
P(x) = a_3x^3 + a_2x^2 + a_1x + a_0 \pmod{p}
$$
The "randomness" of this polynomial means that its coefficients, namely $a_0, a_1, a_2, a_3$, are chosen randomly and uniformly from the set $\{0, 1, ..., p-1\}$. These four coefficients are our **"seed"**.

**3. Generate the Sequence of Values:**
To generate the $i$-th value of the sequence (which we'll call $h(i)$), we simply evaluate the polynomial $P(x)$ at the point $x=i$:
$$
h(i) = P(i) \pmod{p}
$$
We do this for $i$ from $1$ to $n$. This sequence $h(1), h(2), ..., h(n)$ has the property of being 4-wise independent.

---

### Why This Method Works

A fundamental property in mathematics states that a non-zero polynomial of degree $d$ has at most $d$ roots. An important consequence of this property is:
> If we know the values of a degree-3 polynomial at 4 arbitrary points (e.g., $i, j, k, l$), these 4 values are completely independent of each other. Knowing the values of $P(i), P(j), P(k)$ gives us no information about the value of $P(l)$, because for any arbitrary value $y$ for $P(l)$, there is exactly one degree-3 polynomial that passes through these four points.

This guarantees that any group of four generated values is completely independent.

---

### Converting the Output to Bits

The values $h(i)$ that we generated are numbers in the range $[0, p-1]$. To convert them into bits (or into values $\sigma \in \{+1, -1\}$), we can use a simple rule.

* **For bits $\{0, 1\}$**: If $h(i)$ is even, the output is 0; if it's odd, the output is 1.
* **For values $\{-1, +1\}$**: If $h(i) < p/2$, the output is +1; otherwise, the output is -1.

In [3]:

class FourWiseHashFunction:

    def __init__(self, universe_size):
        """
        p must be the first prime number greater than or equal to universe size
        """
        self.p = galois.next_prime(universe_size)
        self.GF = galois.GF(self.p)

        self.a3 = self.GF.Random()  # Generating random coefficients on a finite field GF
        self.a2 = self.GF.Random()
        self.a1 = self.GF.Random()
        self.a0 = self.GF.Random()
        print(f" finit field GF , a0 = {self.a0}. a1 = {self.a1}, a2 = {self.a2}. a3 = {self.a3}")

    def hash(self, item):
        x = self.GF(item)
        evaluation = self.a0 + x * (self.a1 + x * (self.a2 + x * self.a3))
        return 1 if int(evaluation) < self.p / 2 else -1


## Explanation of the Pairwise Independent Value Generation Code

This Python code snippet implements a hash function named `pairwise_hash`, which generates a sequence of **Pairwise Independent** values. The algorithm's core logic is based on expanding a small, random "seed" using the XOR operation.

---

### 1. The Random Seed

* **`k = 10`**: This variable sets the length of the initial seed. With a seed of $k=10$ bits, we can generate up to $2^{10}-1 = 1023$ pairwise independent values.
* **`seed = [...]`**: This line creates a list of `k` random bits (0 or 1). Using the `secrets` module ensures that these initial bits are of high statistical quality.

---

### 2. The Main `pairwise_hash` Function

This function takes an index `i` and returns the corresponding sigma value ($\sigma_i$) based on the seed.

* **Input Validation**:
    First, the function checks if the input index `i` is within the valid range. The valid range for `i` is $1 \le i \le 2^k - 1$. If `i` is outside this range, a `ValueError` is raised.

* **Index-to-Seed Mapping**:
    The line `binary_i = bin(i)[2:].zfill(k)` converts the index `i` into a k-length binary string. This string determines which of the `seed` bits will participate in the XOR operation.

* **XOR Operation**:
    The `for` loop iterates over the binary representation of `i`. If the `j`-th bit of this string is `'1'`, then the `j`-th bit of the `seed` is XORed with the current `output_bit`. The final value is the result of the operation $s_{j_1} \oplus s_{j_2} \oplus \dots$, where $j_x$ are the indices of the '1' bits in the binary representation of `i`.

* **Final Output**:
    Finally, the result of the XOR operation, which is a single bit (0 or 1), is mapped to the values $\{-1, +1\}$. If `output_bit` is 1, the output will be 1; otherwise, it is -1.


In [4]:
k = 7
seed = [secrets.choice([0, 1]) for _ in range(k)]


def pairwise_hash(i, seed):
    if i == 0 or i > 2 ** k - 1:
        raise ValueError("i should be in range [0, 2 ** k - 1]")

    output_bit = 0
    binary_i = bin(i)[2:].zfill(k)

    for j in range(k):
        if binary_i[j] == '1':
            output_bit ^= seed[j]  # XOR operator in python

    return 1 if output_bit == 1 else -1

## Explanation of the Data Streaming Cell

This Python code defines a generator function, `stream_data_from_file`, designed to read numbers from a text file in a memory-efficient way. The function definition itself is placed within a `for` loop that repeats twenty times.

### Core Functionality: `stream_data_from_file`

The primary purpose of this function is to act as a data stream generator. Instead of loading the entire file into memory, it reads and yields one number at a time.

* **Generator-Based Approach**: The function uses the `yield` keyword, which turns it into a generator. This allows it to produce a sequence of values lazily, pausing its state between each value and resuming upon the next request. This is ideal for processing very large files with constant memory usage.

* **Safe File Handling**: It uses the `with open(...)` statement to open the file. This is a best practice in Python as it ensures that the file is automatically and safely closed when the block is exited, even if errors occur.

* **Data Processing and Cleaning**: For each line in the file, the `.strip()` method is called to remove any leading or trailing whitespace (including newline characters). The cleaned line is then converted to an integer via `int()`. The `if cleaned_line:` check ensures that empty lines in the file are skipped.

* **Error Handling**: The function is wrapped in a `try...except` block to gracefully handle two common issues:
    * `FileNotFoundError`: Catches cases where the specified `filepath` does not exist.
    * `ValueError`: Catches cases where a line in the file contains text that cannot be converted to an integer.

In [5]:
def stream_data_from_file(filepath):

    try:
        with open(filepath, 'r') as file:
            for line in file:
                cleaned_line = line.strip()
                if cleaned_line:
                    number = int(cleaned_line)
                    yield number
    except FileNotFoundError:
        print(f" Error : file not found in: {filepath}")
    except ValueError:
        print(f" Error: A line is missing in the file.")


## Explanation of the Final Experiment Code

This Python script executes the core experiment of the assignment. Its main purpose is to empirically compare the performance of three different random hashing strategies (pairwise, 4-wise, and a simulation of full independence) for estimating the second frequency moment ($F_2$) of a data stream.

The experiment is repeated 20 times to allow for an averaged and more stable comparison of the methods.

### 1. Setup and Initialization

```python
file_path = "/datasets/zipf.4.0.txt"

twoWiseTrial = []
fourWiseTrial = []
completelyWiseTrial = []
```

* **`file_path`**: This variable stores the location of the dataset that will be used as the data stream.
* **`...Trial` Lists**: These three lists are initialized to store the final $F_2$ estimate from each of the 20 trials for the three respective methods.

### 2. The Main Experiment Loop

```python
for _ in range(20):
    # ... code for one trial ...
```

This outer loop runs the entire estimation process 20 times. This is crucial for obtaining a reliable result, as a single run of the AMS algorithm can have high variance. By averaging the results of these 20 independent trials, we can get a much better sense of each method's typical accuracy.

### 3. A Single Trial Workflow

Inside the main loop, the following steps are executed for each trial:

* **Data Stream Creation**: `data_stream = stream_data_from_file(file_path)`
    A fresh data stream generator is created from the source file for each run.

* **Hash Function Initialization**:
    ```python
    completely_independent_bits = StatelessConsistentHash()
    fourWise_independent_bits = FourWiseHashFunction(100)
    # The pairwise hash uses a seed defined elsewhere in the code
    ```
    New instances of the hash functions are created for each trial. This is important because it ensures that each of the 20 trials uses a new, independent set of random seeds/polynomials.

* **Counter Initialization**: The accumulator variables `x_twoWiseIndependent`, `x_fourWiseIndependent`, and `x_completelyIndependent` are reset to `0`.

### 4. Stream Processing

```python
for num in data_stream:
    x_twoWiseIndependent += pairwise_hash(num, seed)
    x_fourWiseIndependent += fourWise_independent_bits.hash(num)
    x_completelyIndependent += completely_independent_bits.hash(num)
```

This inner loop iterates through the data stream. For each number, it calls the corresponding hash function for each of the three methods and adds the resulting $\sigma$ value (`+1` or `-1`) to the respective accumulator (`x`).

### 5. Storing the Result of the Trial

```python
twoWiseTrial.append(x_twoWiseIndependent ** 2)
fourWiseTrial.append(x_fourWiseIndependent ** 2)
completelyWiseTrial.append(x_completelyIndependent ** 2)
```

After the entire stream has been processed, the final value of each accumulator `x` is squared to get the $F_2$ estimate for that trial. This estimate is then appended to the corresponding list, storing it for final analysis after all 20 trials are complete. The `average` function defined at the end would then be used on these lists to find the average estimate for each method.

In [6]:
file_path = "/datasets/zipf.4.0.txt"

twoWiseTrial = []
fourWiseTrial = []
completlyWiseTrial = []

for _ in range(20):

    data_stream = stream_data_from_file(file_path)

    completly_independent_bits = StatelessConsistentHash()
    fourWise_independent_bits = FourWiseHashFunction(int(100))

    x_twoWiseIndependent = 0
    x_fourWiseIndependent = 0
    x_completlyIndependent = 0

    for num in data_stream:
        x_twoWiseIndependent += pairwise_hash(num, seed)
        x_fourWiseIndependent += fourWise_independent_bits.hash(num)
        x_completlyIndependent += completly_independent_bits.hash(num)

    twoWiseTrial.append(x_twoWiseIndependent ** 2)
    fourWiseTrial.append(x_fourWiseIndependent ** 2)
    completlyWiseTrial.append(x_completlyIndependent ** 2)


def average(list):
    return sum(list) / len(list)

 finit field GF , a0 = 90. a1 = 68, a2 = 40. a3 = 62
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 21. a1 = 52, a2 = 7. a3 = 91
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 61. a1 = 36, a2 = 18. a3 = 64
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 40. a1 = 71, a2 = 50. a3 = 41
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 82. a1 = 83, a2 = 85. a3 = 20
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 29. a1 = 41, a2 = 27. a3 = 99
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 77. a1 = 74, a2 = 54. a3 = 50
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 52. a1 = 62, a2 = 10. a3 = 22
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 91. a1 = 61, a2 = 53. a3 = 30
 Error : file not found in: /datasets/zipf.4.0.txt
 finit field GF , a0 = 48. a1 = 6, a2 = 28. a3 = 39
 Error : file

In [7]:
data_stream = stream_data_from_file(file_path)

frequencies = defaultdict(int)
for number in data_stream:
    frequencies[number] += 1

print(f"item frequencies : {dict(frequencies)}")

real_F2 = 0
for count in frequencies.values():
    real_F2 += count ** 2

 Error : file not found in: /datasets/zipf.4.0.txt
item frequencies : {}


In [8]:
print(f" tow wise independent : {average(twoWiseTrial)}")
print(f" four wise independent : {average(fourWiseTrial)}")
print(f" completly independent : {average(completlyWiseTrial)}")
print(f"real F2 : {real_F2}")

 tow wise independent : 0.0
 four wise independent : 0.0
 completly independent : 0.0
real F2 : 0


In [9]:
headers = [ "name file", "F2: Two Wise Independent", "F2: Four Wise Independent",
            "F2: Completly Independent", "F2 :Real F2", "n_parameter in Four wise class :",
            "k parameter in Two wise function :"]
data = [["zipf.1.5.", "4155349444.0", "1894836613.0", "1456452010.0", "1772040544", "1000000000", "30" ],
        ["zipf.2.0.", "5712034084.0", "3881677450.0", "3690387011.8", "4001451216", "100000", "20" ],
        ["zipf.2.", "2935689124.0", "5037947196.8", "4855460405.4", "4001256520", "100000", "15" ],
        ["zipf.3.0.", "8299574404.0", "6999677815.6", "7104513458.4", "7015960234", "1000", "10"],
        ["zipf.4.0.", "9463009284.0", "8407260579.6", "8827142045.2", "8578303586", "100", "7" ]]
print(tabulate(data, headers, tablefmt="fancy_grid"))
print("\n")

╒═════════════╤════════════════════════════╤═════════════════════════════╤═════════════════════════════╤═══════════════╤════════════════════════════════════╤══════════════════════════════════════╕
│ name file   │   F2: Two Wise Independent │   F2: Four Wise Independent │   F2: Completly Independent │   F2 :Real F2 │   n_parameter in Four wise class : │   k parameter in Two wise function : │
╞═════════════╪════════════════════════════╪═════════════════════════════╪═════════════════════════════╪═══════════════╪════════════════════════════════════╪══════════════════════════════════════╡
│ zipf.1.5.   │                4.15535e+09 │                 1.89484e+09 │                 1.45645e+09 │    1772040544 │                         1000000000 │                                   30 │
├─────────────┼────────────────────────────┼─────────────────────────────┼─────────────────────────────┼───────────────┼────────────────────────────────────┼──────────────────────────────────────┤
│ zipf.2.0.   │